<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/ClassificationInvoices_Tf_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip uninstall keras-preprocessing

In [ ]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

In [17]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report,confusion_matrix
import warnings
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [18]:
my_data_dir = 'data'

# CONFIRM THAT THIS REPORTS BACK 'test', and 'train'
os.listdir(my_data_dir) 

test_path = my_data_dir+'\\test\\'
train_path = my_data_dir+'\\train\\'

os.listdir(test_path)
os.listdir(train_path)

['invoices', 'others']

In [19]:
#Preparing the Data for the model
#Image Manipulation

image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [21]:
#Generating many manipulated images from a directory
image_gen.flow_from_directory(train_path)
image_gen.flow_from_dirctory(test_path)

Found 3466 images belonging to 2 classes.


AttributeError: 'ImageDataGenerator' object has no attribute 'flow_from_dirctory'

In [8]:
#https://stats.stackexchange.com/questions/148139/rules-for-selecting-convolutional-neural-network-hyperparameters
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary so we use sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


NameError: name 'image_shape' is not defined

In [ ]:
#Early Stopping


early_stop = EarlyStopping(monitor='val_loss',patience=2)
help(image_gen.flow_from_directory)
batch_size = 16

train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

train_image_gen.class_indices
warnings.filterwarnings('ignore')

results = model.fit_generator(train_image_gen,epochs=20,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

model.save('malaria_detector.h5')

#Evaluating the Model
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()
model.metrics_names
model.evaluate_generator(test_image_gen)
# https://datascience.stackexchange.com/questions/13894/how-to-get-predictions-with-predict-generator-on-streaming-test-data-in-keras
pred_probabilities = model.predict_generator(test_image_gen)
pred_probabilities
predictions = pred_probabilities > 0.5
# Numpy can treat this as True/False for us
predictions

print(classification_report(test_image_gen.classes,predictions))

#Predicting on an Image
# Your file path will be different!
para_cell

my_image = image.load_img(para_cell,target_size=image_shape)

my_image = image.img_to_array(my_image)

type(my_image)

my_image.shape

my_image = np.expand_dims(my_image, axis=0)

my_image.shape

model.predict(my_image)

train_image_gen.class_indices

test_image_gen.class_indices
type(my_image)